### 데이터 로드 및 전처리
- reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다. 데이터셋은 `keras.datasets`에서 불러올 수 있는 reuters 데이터 셋을 활용하고, 이때 파라메터 조정을 통해 단어의 개수는 10000개로 제한합니다(num_words=10000).
- 데이터셋 분리 작업에서는 sklearn 라이브러리의 `train_test_split` 함수를 활용합니다.
- 훈련, 검증, 테스트 데이터와 레이블 종류가 몇개인지 출력합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from tensorflow.keras import models, layers
from tensorflow.keras import callbacks
from tensorflow.keras import utils
from keras.datasets import reuters

In [ ]:
# 문제 2-1: reuters 데이터셋을 로드하고, 훈련 데이터셋에서 25%를 검증 데이터셋으로 분리합니다.
(train_data, val_data), (test_data, train_labels) = reuters.load_data(num_words=10000, test_split=0.25)


# 훈련, 검증, 테스트 데이터와 레이블 종류가 몇 개인지 출력합니다.
print(len(train_data))
print(len(val_data))
print(len(test_data))
print(len(set(train_labels)))

- 텍스트 데이터의 전처리를 위해서 각 데이터셋마다 원-핫 인코딩을 수행합니다. 원-핫 인코딩 함수를 직접 만들어보세요.
- 레이블 데이터들은 범주형 형태로 to_categorical() 함수를 사용해 변환합니다.

In [ ]:
# 문제 2-2: 텍스트 데이터의 전처리를 위해서 각 데이터셋마다 원-핫 인코딩을 수행합니다.
def one_hot_encoding(data, dim=10000):
  from tensorflow.keras.preprocessing.sequence import pad_sequences
  # sentence slicing within max_len
  max_len = 100

  if type(data[0]) == list:
    data = pad_sequences(data, maxlen=max_len)
  else:
    data = utils.to_categorical(data)
  
  # print(data[0])
  # print(type(data[0]))
  return data

x_train = one_hot_encoding(train_data)
x_val = one_hot_encoding(val_data)
x_test = one_hot_encoding(test_data)

# 레이블 데이터들은 범주형 형태로 to_categorical() 함수를 사용해 변환합니다.
y_test = one_hot_encoding(train_labels)

### 모델 구성 및 컴파일
- BatchNormalization과 Dropout을 적용하여 빠르게 학습하면서 과대적합을 방지하고, 46개의 뉴스 주제를 분류하는 딥러닝 모델을 구성합니다.

In [ ]:
# 문제 2-3: 빠른 학습과 과대적합을 방지하기 위해 BatchNormalization과 Dropout을 적용한 딥러닝 모델을 구성합니다.
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
# Batch Normalization 추가
model.add(layers.BatchNormalization())
# Dropout 추가
model.add(layers.Dropout(0.7))
model.add(layers.Dense(64, activation='relu'))
# Batch Normalization 추가
model.add(layers.BatchNormalization())  # Batch Normalization 추가
# Dropout 추가
model.add(layers.Dropout(0.7))
model.add(layers.Dense(46, activation='softmax'))

In [ ]:
# 모델 요약
model.summary()
# 모델 과정 그림으로 표현하기
utils.plot_model(model, show_shapes=True)

### 모델 학습
- ModelCheckpoint와 EarlyStopping 콜백 함수를 적용하여 모델 학습을 진행합니다.

In [ ]:
# 문제 2-4: ModelCheckpoint와 EarlyStopping 콜백 함수를 적용하여 모델 학습을 진행합니다.
# model compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# EarlyStopping
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3)
# Modelcheckpoint
model_checkpoint = callbacks.ModelCheckpoint(filepath='mskim_test.h5', monitor='val_loss', save_best_only=True, verbose=1)


# 모델 학습
history = model.fit(x_train, y_train, batch_size=512, epochs=10, validation_data = (x_val, y_val), callbacks=[early_stopping, model_checkpoint])

In [ ]:
## 위 코드에서 다음 에러 발생 중
"""
ValueError: Data cardinality is ambiguous:
  x sizes: 8421
  y sizes: 6736
Make sure all arrays contain the same number of samples.
"""

# 추후 수정 필요
## 데이터 전처리에서 테스트하다가 꼬임

- 학습 히스토리의 `loss`, `val_loss`, `accuracy`, `val_accuracy`를 차트로 시각화합니다.

In [ ]:
# 학습 히스토리의 `loss`, `val_loss`, `accuracy`, `val_accuracy`를 차트로 시각화합니다.
plt.style.use('seaborn-white')

history_dict = history.history

# for key in history_dict:
#     print(key)

loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(epochs, loss, 'b-', label='train_loss')
ax1.plot(epochs, val_loss, 'r-', label='val_loss')
ax1.set_title('Train and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.grid()
ax1.legend()

accuracy = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(epochs, accuracy, 'b-', label='train_accuracy')
ax2.plot(epochs, val_accuracy, 'r-', label='val_accuracy')
ax2.set_title('Train and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.grid()
ax2.legend()

plt.show()

### 모델 평가 및 예측
- 테스트 데이터셋을 이용해 모델을 평가합니다.

In [ ]:
# 문제 2-5: 테스트 데이터셋을 이용해 모델을 평가합니다.
results = model.evaluate(x_test, y_test, batch_size=128)